### Linear Time Selection

In this notebook we will look at two types of selection algorithms, the RSelect, or the randomized selection algorithm and a deterministic version called DSelect. But before we proceed the question is "What is selection?" A selection problem aims to find the $i^{th}$ smallest number for n given numbers. The simplest approach would be to sort the input array and then select the $i^{th}$ position number essentially doing the selection in $nlogn$ time. This however is not the optimum way as selection if an easier problem than sorting and thus we should achieve this in much less running time than sorting. To select a number from n given numbers, we will have to view all the numbers at the minimum and thus $O(n)$ is a reasonable lower bound and we know that sorting can be done in $O(nlogn)$ which is an upperbound for our problem.

Can we do something to possibly perform the selection in $O(n)$ time? Let us implement the selection problem using the partition routine we used in quick sort.



In [1]:
def swap(in_array, ix1, ix2):
    t = in_array[ix1]
    in_array[ix1] = in_array[ix2]
    in_array[ix2] = t
    
def partition(in_array, start_ix, end_ix):
      
    partition_idx = start_ix
    i = start_ix
    pivot = in_array[partition_idx]
    for idx in range(start_ix + 1, end_ix):
        if in_array[idx] <=  pivot:
            i += 1
            if idx != i:
                swap(in_array, i, idx)

    swap(in_array, start_ix, i)
    partition_idx = i
    return partition_idx

def RSelect(array, i, start_ix = 0, end_ix = -1):
    #From a given array, find the ith order statistic
    end_ix = len(array) if end_ix == -1 else end_ix    
    pivot_idx  = partition(array, start_ix, end_ix)    
    stat = pivot_idx - start_ix + 1
    
    if i == stat:
        return array[pivot_idx]
    elif i < stat:
        return RSelect(array, i, start_ix, pivot_idx)
    else:
        return RSelect(array, i - stat, pivot_idx + 1, end_ix)

In [2]:
print('1st Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 1))
print('2nd Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 2))
print('3rd Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 3))
print('4th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 4))
print('5th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 5))
print('6th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 6))
print('7th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 7))
print('8th Order stat is', RSelect([3, 8, 2, 5, 1, 4, 7, 6], 8))

1st Order stat is 1
2nd Order stat is 2
3rd Order stat is 3
4th Order stat is 4
5th Order stat is 5
6th Order stat is 6
7th Order stat is 7
8th Order stat is 8


We will test the implementation with some of the test data given [here](http://theory.stanford.edu/~tim/algorithmsilluminated.html)

In [3]:
with open('problem6.5test1.txt', 'r') as f:
    linest1 = [int(line.strip()) for line in f.readlines()]

with open('problem6.5test2.txt', 'r') as f:
    linest2 = [int(line.strip()) for line in f.readlines()]

print('5th Order statistic of the content expected to be 5469, got', RSelect(linest1, 5))
print('50th Order statistic of the content expected to be 4715, got', RSelect(linest2, 50))

5th Order statistic of the content expected to be 5469, got 5469
50th Order statistic of the content expected to be 4715, got 4715




From above tests, the randomized implementation works in place with constant extra memory and works on a similar way quick sort does. The only difference is that after partition we either recurse on the left or the right side of the array unlike sorting where we recurse on both partitions around pivot. We will prove that the above algorithm runs in linear time.

---
**Intutition**

The partition sub routine does exactly what it does in case of quick sort. Once we choose the partition routine, we know in linear time, what position the pivot element will end up in after the recursve calls. Once we know that value, we can then recurse on the left or the right split after the pivot. In the best case suppose we end up picking the median of the input array as pivot, in which case the array will be split in exactly halves. Since we always do linear work outside the recursive calls, we can express our recurrance as follows
$$T(n)\:=\:T(\frac{n}{2}) + O(n) $$

By master method, a = 1, b = 2 and d = 1, we have $a < b^d$ which is case 2 of the master method. The running time is in this case dominated by the work done outside recursive calls giving us the time as $O(n^d)$ = $O(n)$ in this case. This shows us that if we get an approximate median and split the array into two approximately equal pieces, we can expect the selection problem to run in linear time. Let us see how exactly we get linear time on an average for a given array using the following proof.

---
**Proof**

We will track the progress of our selection problem in phases. we will call the problem is in phase j if it is operating on an array of length $(\frac{3}{4})^{j + 1}$ to $(\frac{3}{4})^{j}$. For example, it will be in phase 0, if the size of the array is between 0.75n to n, phase 1 of the size array is 0.56n to 0.75n and so on. The maximum value of phase j when the value of the input size will be 1 should be $log_{4/3}n$.

We will now define a random variable $X_j$ which gives is the number of times the selection process made recursive calls in phase j. The minimum value of this variable is 0.

The maximum size of the array in phase j is $(\frac{3}{4})^j$ and thus the work done in this phase is no more than $c\cdot (\frac{3}{4})^j \cdot n$. Thus

$$Running\:time\:of\:RSelect\:is\:\leq \sum_{j \geq 0} c\cdot (\frac{3}{4})^j\cdot n X_j$$

By linearity of expectations

$$E[Running\:time\:of\:RSelect\:is]\:\leq cn \sum_{j \geq 0} (\frac{3}{4})^jE[X_j]$$

How do we find $E[X_j]$?

Whenever we choose an approximate median as pivot, a number that gives us 25-75% split of the input array, we are guaranteed to procees to next phase. If only when the split produces an array giving is greater than 75% of elements remaining after partition, we stay in the phase j for another recursive call. Thus we can conclude, picking an approximate median is guaranteed to take us to next phase.

Secondly, picking an approximate median has a probability of 50%. Suppose we have n numbers say, 1 to 100, then by picking anything from 26 to 75 is an approximate median which makes up 50% of the input numbers.

The random variable $X_j$ is similar to a coin flipping problem. Suppose N is a random variable which counts the number of times we need to flip the coin to get heads (or tails).

The random variable $X_j$ and $N$ are similar but the $E[X_j] \leq E[N]$

- The coin flopping experiment has to flip the coin atleast once to get heads. The value of X_j can be 0 if we completely skip the phase.
- We have maximum probability of 0.5 of not going out of phase j and staying in the same phase. This is similar to 0.5 probability we get tails and we we need to flip the coin another time.

The expected value of the coin flip experiment is 

$E(N)\: = \: 1\: +\: \frac{1}{2}E(N)$

The only value of E(N) that satisfies the above equation is 2. The equation is explained as follows. The value 1 is for that minimum one flip needed and the value 0.5 is the probability that we get tails. We can thus say that $E(X_j) \leq 2$

Geometric sequence $1\: + \: r^2\: + \: r^3 \dots r^k = \frac{1 - r^{k + 1}}{1 - r}$

This $\sum_{j \geq 0} (\frac{3}{4})^j\: \leq\: \frac{1}{1 - \frac{3}{4}} = 4$

Therefore, 
$E[Running\:time\:of\:RSelect\:is]\:\leq cn \sum_{j \geq 0} (\frac{3}{4})^jE[X_j] = 4\cdot 2cn = 8cn$

Hence, the running time of randomized select algorithm on an average runs in $O(n)$

---

Next we will look at an algorithm which does the selection in linear time and is deterministic. Its called **DSelect** in our notebook

We will analyze the running time of the algorithm. For Pseudo code of the algorithm refer page 169 of the book.

The first time consuming activity of DSelect is to find the median of medians. For this purpose we find the break  the input array in batches of 5 and find medians of each of these splits of 5. This way we find the a total of $\frac{n}{5}$ medians. We recursively keep finding the medians of medians till we hit the base case where we only one element in the input array. To illustrate this, consider we have the following input array

11, 6, 10, 2, 15, 8, 1, 7, 14, 3, 9, 12, 4, 5, 13

We break find medians of the splits of 5 of the array giving us 10 (median of 11, 6, 10, 2, 15), 7 (median of 8, 1, 7, 14, 3, 9) and 9 median of (9, 12, 4, 5, 13). Median of these three medians is 9 and thus this is the pivot of our choice.

Finding median of 5 numbers is a constant time activity and let that constant be c. We operate on an array of $\frac{n}{5}$ to find medians. Thus the time taken to find the medians of medians is $\frac{n}{5}\cdot c$ = $O(n)$

Thus find median of medians is a Linear time operation.

The median of medians is found by calling DSelect recursively on an array of size $\frac{n}{5}$ and then the partition, which is done in linear time, determines the statistic of the pivot element (the median of medians). Based on the $i{th}$ order statistic we are interested in, we either recurse on the portion of the input to the left of the pivot or the one on the right. We therefore have two sub problems

1. The Dselect called on $\frac{n}{5}$ array to find median of medians
2. The second recursive call on DSelect on the reduced array.

Outside the recursive call, the DSelect does linear work to find the medians and to partition the numbers around pivot. Therefore the recurrence is 

$T(n)\: \leq \: T(\frac{n}{5})\:+\:T(?)\:+ O(n)\:$

Right now we are interested in deterministically find $T(?)$

For this purpose we introduce the 30-70 Lemma (which we will prove later). This Lemma states that the median of medians is no less than at least 60% of elements in at least 50% of the groups of 5 and no greater than at least 60% of at least 50% of groups of 5. The value 60% of 50% is 0.6 * 0.5 = 30%. This sounds confusing, but it essentially says that there are at least 30% numbers in the input array those are smaller than the partition element (median of medians), and there are at least 30% of numbers in the input array those are larger than the partition element. The result of this is, that under no circumstance we will get the split around partition greater than 70% of original input. Therefore, $T(?) \leq T(\frac{7n}{10})$ hence the running time of DSelect is 

$T(n)\: \leq \: T(\frac{n}{5})\:+\:T(\frac{7n}{10})\:+ O(n)\:$

We will prove the algorithm runs in linear time using induction. We assume that for k < n, $T(k) \leq lk$

We will assign l to some arbritrary constant independent of n, for our derivation we will assign l = 10c. This is a legitimate assumption as for the base case we know T(1) = 1. Since $c\geq 1$, $T(1) \leq 10c$

Therefore for $n\geq 2$

$T(n)\: \leq \: l\frac{n}{5}\:+\:l\frac{7n}{10}\:+ cn\: = \frac{9ln}{10}\:+ cn\: = 9cn + cn = 10cn = l\cdot n$

This proves the indictive step that the $T(n)\: \leq l\cdot n\:=\:O(n)$


We are still to prove the 30-70 Lemma. The simple explanation is as follows.

- Consider k number arrays each of length 5. For simplicity lets assume k is 5. Thus there are two medians smaller and two medians larger than the median or medians (mom). We call them $s_1$, $s_2$ and $l_1$, $l_2$ respectively.
- We can therefore conclusively say that no more than 3 of 5, which is 60% of numbers including the medians in $s_1$ and $s_2$ and the minimum 2 numbers in the same batch of 5 as mom are smaller than mom.
- Similarly, we can therefore conclusively say that no more than 3 of 5, which is 60% of numbers including the medians in $l_1$ and $l_2$ and the maximum 2 numbers in the same batch of 5 as mom are larger than mom.
- Thus for this example where k = 5 (n = 25). Minimum 8 numbers are smaller than mom and minimum 8 numbers are larger than mom. The next recursive call will not have the pivot and either of these 8 numbers which makes it 9 numbers minimum. This means the maximum size of the array we can have is 16 which is 64% of input array.

This generalization holds true for any large value of k and we are guaranteed to not have more than 70% of original input passed to the subsequent recursive call.

TODO: Complete the implementation below.


In [4]:

#Finds index of median in constant time in the context of DSelect as the size of array is maximum 5,
#assuming that we have the array of
def median_ix(array):
    med = sorted(array)[len(array) // 2]
    return [i for i in range(0, len(array)) if array[i] == med][0]

def median_of_medians(in_array, start_ix, end_ix):
    splits = [(in_array[i:min(i + 5, end_ix)], median_ix(in_array[i:min(i + 5, end_ix)])) for i in range(start_ix, end_ix, 5)]
    if len(splits) > 1:
        median, median_idx = median_of_medians([split[i] for (split, i) in splits], 0, len(splits))
    else:        
        split = splits[0]
        return split[0][split[1]], start_ix + split[1]
        
    array_split, i = splits[median_idx]
    return array_split[i], start_ix + i + median_idx * 5


def DSelect(in_array, i, start_ix = 0, end_ix = -1):
    end_ix = len(in_array) if end_ix == -1 else end_ix  
    #Note that we dont really use DSelect recursively to reduce the complexity of the code, instead we recursively
    #Call another function. The running time analysis however stays the same
    median_of_median, ix = median_of_medians(in_array, start_ix, end_ix)
    swap(in_array, start_ix, ix)
    pivot_idx = partition(in_array, start_ix, end_ix)
    stat = pivot_idx - start_ix + 1

    if i == stat:        
        return in_array[pivot_idx]
    elif i < stat:
        return DSelect(in_array, i, start_ix, pivot_idx)
    else:
        return DSelect(in_array, i - stat, pivot_idx + 1, end_ix)


In [5]:
print('1st Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 1))
print('2nd Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 2))
print('3rd Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 3))
print('4th Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 4))
print('5th Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 5))
print('6th Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 6))
print('7th Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 7))
print('8th Order stat is', DSelect([3, 8, 2, 5, 1, 4, 7, 6], 8))

1st Order stat is 1
2nd Order stat is 2
3rd Order stat is 3
4th Order stat is 4
5th Order stat is 5
6th Order stat is 6
7th Order stat is 7
8th Order stat is 8


In [6]:
with open('problem6.5test1.txt', 'r') as f:
    linest1 = [int(line.strip()) for line in f.readlines()]

with open('problem6.5test2.txt', 'r') as f:
   linest2 = [int(line.strip()) for line in f.readlines()]

print('5th Order statistic of the content expected to be 5469, got', DSelect(linest1, 5))
print('50th Order statistic of the content expected to be 4715, got', DSelect(linest2, 50))

5th Order statistic of the content expected to be 5469, got 5469
50th Order statistic of the content expected to be 4715, got 4715


With the above test cases giving right answers, lets execute the code using numbers created using 100,000 and 1,000,000 digits of pi. Each number is 10 consecutive numbers of pi. Let us construct this list using the files containing 100,000 and 1,000,000

In [7]:
def load_numbers_from_file(file):
    with open(file) as f:
        lines = f.readlines()

    pistr = lines[0].replace('.', '', 1)
    nums = [int(pistr[i:i + 10]) for i in range(0, len(pistr), 10)]

    return nums

In [8]:
pi100000num = load_numbers_from_file('pi100000.txt')
med_idx_1 = len(pi100000num) // 2
true_median1 = sorted(pi100000num)[med_idx_1]
rselect_median_1 = RSelect(pi100000num, med_idx_1 + 1)
pi100000num = load_numbers_from_file('pi100000.txt')
dselect_median_1 = DSelect(pi100000num, med_idx_1 + 1)
print('Using 100000 digits of pi, true median is', true_median1, 'RSelect gave', rselect_median_1, 'DSelect gave', dselect_median_1)


pi1000000num = load_numbers_from_file('pi1000000.txt')
med_idx_2 = len(pi1000000num) // 2
true_median2 = sorted(pi1000000num)[med_idx_2]
rselect_median_2 = RSelect(pi1000000num, med_idx_1 + 1)
pi1000000num = load_numbers_from_file('pi1000000.txt')
dselect_median_2 = DSelect(pi1000000num, med_idx_2 + 1)
print('Using 1000000 digits of pi, true median is', true_median2, 'RSelect gave', rselect_median_2, 'DSelect gave', dselect_median_2)

Using 100000 digits of pi, true median is 5048893989 RSelect gave 5048893989 DSelect gave 5048893989
Using 1000000 digits of pi, true median is 5013922558 RSelect gave 503172651 DSelect gave 5013922558


TODO: RSelect is not giving correct results in the final case and needs to be solved.